In [ ]:
import pandas as pd
from pathlib import Path

project_root = Path.cwd().parent
data_path = project_root / "data" / "processed"
de_price_file = data_path / "DE_spot_prices.csv"
hu_price_file = data_path / "HU_spot_prices.csv"

if not de_price_file.exists() or not hu_price_file.exists():
    raise FileNotFoundError("Price files not found.")
else:
    de_prices = pd.read_csv(de_price_file, index_col=0, parse_dates=True)
    de_prices.columns = ['DE_Price']

    hu_prices = pd.read_csv(hu_price_file, index_col=0, parse_dates=True)
    hu_prices.columns = ['HU_Price']

    de_prices.index = de_prices.index.tz_convert('UTC')
    hu_prices.index = hu_prices.index.tz_convert('UTC')

    merged_prices = de_prices.join(hu_prices, how='outer')
    merged_prices['Price_Delta_HU_DE'] = merged_prices['HU_Price'] - merged_prices['DE_Price']

    print("Data fetching and preprocessing complete.")